In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import regex as re
from lxml import etree as et
from itertools import repeat
import csv
import random
import time
from tqdm import tqdm_notebook # tqdm_notebook es una herramienta que permite mostrar barra de progreso al trabajar con bucles grandes
import warnings
warnings.filterwarnings('ignore')
import math

# **WEB SCAPING BACKLOGGD**

En este fichero se puede ver el código utilizado para realizar el web scrpaing con el que he generado mi propio dataset.

Aunque me basé en un script existente, esta versión es mucho más avanzada, ya que está pensada para la recopiación de una gran cantidad de datos. Por ello, controla todo tipo de excepciones, recopila más datos y es más eficiente.

El script base se encuentra en este misma carpeta, se llama 'web_scaping_base.py'. Revisa, compara y luego me cuentas :)

In [ ]:
'''
Un agente de usuario es una cadena que identifica la aplicación y la versión del software que está realizando la solicitud a un servidor web. 
Se utiliza para enviar solicitudes HTTP a la página web que estamos extrayendo.
Cada elemento de la lista header_list representa un agente de usuario simulado para un navegador web específico. 
El objetivo de usar múltiples agentes de usuario es simular el comportamiento de diferentes navegadores y evitar ser bloqueado por el servidor debido a solicitudes automatizadas o scraping.
Esta parte del código ayuda a realizar scraping de manera más amigable y a evitar posibles bloqueos por parte del servidor
'''

In [2]:
# Lista de agentes de usuario
header_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,                           like Gecko) Chrome/103.0.5060.66 Safari/537.36",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393"]

# URL base para la página de juegos
base_url = 'https://www.backloggd.com/games/lib/popular?page='
games_list = []
d = 0

In [3]:
# Esta sección del código me ayuda a determinar la última página que pude recopilar antes de que petara el scraping por alguna razón
# IMPORTANTE --> cada vez que sufra un pete actualizar los datos AQUÍ para que el bucle no vuelve a añadir datos que ya tengo en el DF

# Número total de juegos recopilados (lo miro a través de .info())
total_juegos_recopilados = 52037

# Número de juegos por página (cada página de la web tiene 6 filas x 6 columnas)
juegos_por_pagina = 36

# Calcular la última página exitosa
ultima_pagina_exitosa = math.ceil(total_juegos_recopilados / juegos_por_pagina)

print(f"La última página exitosa es: {ultima_pagina_exitosa}")

La última página exitosa es: 1446


In [4]:

# Guardar el número de la última página exitosa en un archivo, de esta manera, en el siguiente paso puedo indicarle en el bucle desde dónde tengo que empezar y además no se me olvida ya que está guardado
with open('ultima_pagina_exitosa_links.txt', 'w') as file:
    file.write(str(ultima_pagina_exitosa))


In [5]:
# Verificar si existe un archivo que contiene la última página exitosa
try:
    with open('ultima_pagina_exitosa_links.txt', 'r') as file:
        last_successful_page_links = int(file.read())
except FileNotFoundError:
    last_successful_page_links = 0

# Incrementar en 1 la última página exitosa para comenzar desde la siguiente
#last_successful_page_links += 1


# Lista para almacenar enlaces de juegos
game_links = []

for page_no in tqdm_notebook(range(last_successful_page_links, 1500)): # en range se incia desde la última página guardada con éxito hasta la página que seleccione (la última es 4510)
    page_url = base_url + str(page_no)
    #print(page_url)
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(page_url, headers = header)
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        g = soup2.find('div', {'class' : 'row show-release toggle-fade'})
        games = g.find_all('div', {'class' : 'col-2 my-2 px-1 px-md-2'})
        game_ref = [game.find('a').get('href') for game in games]
        game_links.extend(['https://www.backloggd.com' + ref for ref in game_ref])

  0%|          | 0/54 [00:00<?, ?it/s]

In [6]:
# check del número de resultados (links de juegos) que se han scrapeado y almacenado correctamente
print(len(game_links))

1944


In [7]:
# Definir las columnas para el DataFrame
cols = ['Title', 'Release Date', 'Team', 'Rating', 'Times Listed', 'Number of Reviews', 'Genres', 'Summary', 'Reviews','Platforms', 'Plays', 'Playing', 'Backlogs', 'Wishlist']
backloggd = pd.DataFrame(columns=cols) # en este dataframe se recogen los datos cada vez que se hace web scraping

In [8]:
# Intentar cargar el progreso anterior
#try:
#backloggd_progreso = pd.read_csv('backloggd_progreso.csv')
#game_links = backloggd_progreso['Link'].tolist()  # Ajusta el nombre de la columna según corresponda
#except FileNotFoundError:
#game_links = []

# Iterar sobre los enlaces de juegos para obtener información detallada
for link in tqdm_notebook(game_links): # tqdm_notebook es una herramienta que permite mostrar barra de progreso al trabajar con bucles grandes
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(link, headers = header)
     # Verificar si la página se cargó correctamente
    if webpage.status_code == 200:
        soup1 = BeautifulSoup(webpage.content, 'html.parser')
        soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
        
       
        # Obtener información básica del juego (título, fecha de lanzamiento, equipos, calificación, etc.)
        title_element = soup2.find('div', {'class': 'col-auto pr-1'})
        title = title_element.get_text().strip() if title_element else None

        release_date_element = soup2.find('div', {'class': 'col-auto mt-auto pr-0'})
        release_date = ' '.join(release_date_element.get_text().strip().split()[-3:]) if release_date_element else None

        

        try:
            teams = soup2.find('div', {'class' : 'col-auto pl-lg-1 sub-title'})
            teams = teams.find_all('a')
            teams = [i.get_text().strip() for i in teams]
        except:
            teams = None
            #print(teams)
        
        try:
            rating = float(soup2.find(id = 'score').get_text().strip()[-3:])
        except:
            rating = np.nan

        table = soup2.find_all('div', {'class' : 'col-12 mb-1'})#.get_text().strip()
        feats = [f.get_text().strip().split('\n')[0].strip() for f in table] # feats contiene las características de la tabla
        # las variables 'Plays', 'Playing', 'Backlogs', y 'Wishlist' se llenan con los valores de la lista results
        results = [r.get_text().strip().split('\n')[-1].strip() for r in table] # results recoge la info
        #print(feats,results)



        # dicted = {}
        # for i in range(len(feats)):
        #     dicted[feats[i]] = results[i]
        
        try:
            nlists = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]
        except:
            nlists = None
        #nlists = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]
        
        #nreviews = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]

        try:
           nreviews = soup2.find('p', {'class' : 'game-page-sidecard'}).get_text().strip().split()[0]
        except:
            nreviews = None 

        genres = soup2.find_all('p',{'class' : 'genre-tag'})
        genres = [genre.get_text().strip() for genre in genres]

        try:
            summary = soup2.find(id='collapseSummary').get_text().strip()
        except:
            summary = None

        #summary = soup2.find(id = 'collapseSummary').get_text().strip()

        try:
            review_section = soup2.find(id = 'game-reviews-section')
            reviews = review_section.find_all('div', {'class' : 'row pt-2 pb-1 review-card'})
            reviews = [r.find('div', {'class' : 'formatted-text'}).get_text().strip() for r in reviews]
        except:
            reviews = None

        try:
            platform_tags = soup2.find_all('a', class_='game-page-platform')
            platforms = ', '.join([platform.get_text(strip=True) for platform in platform_tags])
            platforms_row = [platforms]
        except:
            platforms_row = None

        # 'row' contiene la lista de variables con la información correspondiente de cada variable hasta 'platform'
        # IMPORTANTE --> si se quieren añadir nuevas variables, colocarlas en la lista de la variable 'cols' de arriba del todo antes que 'plays' para que no se solapen los datos
        row = [title, release_date, teams, rating, nlists, nreviews, genres, summary, reviews, platforms_row]
        row.extend(results) # las variables 'Plays', 'Playing', 'Backlogs', y 'Wishlist' se llenan con los valores de la lista results
        backloggd.loc[len(backloggd.index)] = row # los datos recopilados se añaden a un dataframe que se encuentra en la variable 'backloggd'

        '''
        La razón por la que no se han agregado excepciones para results, genres, summary, reviews, y platforms es porque 
        en esos casos se han utilizado listas generadas a partir de búsquedas en el HTML (find_all)
        Si la búsqueda no encuentra ningún elemento, simplemente se obtiene una lista vacía ([]), y no se producirían excepciones de tipo AttributeError o TypeError
        '''

  0%|          | 0/1944 [00:00<?, ?it/s]

In [9]:
# El DF base se llama 'existing_data'

# Cargar el DataFrame existente desde el archivo CSV
existing_data = pd.read_csv('backloggd_dataset.csv')

# Concatenar el DataFrame existente con los últimos datos cargados en el otro DataFrame (backloggd)
existing_data = pd.concat([existing_data, backloggd], ignore_index=True)

# Guardar el DataFrame actualizado en el archivo CSV
existing_data.to_csv('backloggd_dataset.csv', index=False)


In [10]:
# Aquí se puede ver los datos de la última carga
backloggd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1944 entries, 0 to 1943
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1944 non-null   object 
 1   Release Date       1944 non-null   object 
 2   Team               1100 non-null   object 
 3   Rating             600 non-null    float64
 4   Times Listed       1944 non-null   object 
 5   Number of Reviews  1944 non-null   object 
 6   Genres             1944 non-null   object 
 7   Summary            1944 non-null   object 
 8   Reviews            1944 non-null   object 
 9   Platforms          1944 non-null   object 
 10  Plays              1944 non-null   object 
 11  Playing            1944 non-null   object 
 12  Backlogs           1944 non-null   object 
 13  Wishlist           1944 non-null   object 
dtypes: float64(1), object(13)
memory usage: 227.8+ KB


In [11]:
# el total de datos scrapeados
existing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53981 entries, 0 to 53980
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              53981 non-null  object 
 1   Release Date       53981 non-null  object 
 2   Team               36125 non-null  object 
 3   Rating             22920 non-null  float64
 4   Times Listed       53981 non-null  object 
 5   Number of Reviews  53981 non-null  object 
 6   Genres             53981 non-null  object 
 7   Summary            49586 non-null  object 
 8   Reviews            53981 non-null  object 
 9   Platforms          53981 non-null  object 
 10  Plays              53981 non-null  object 
 11  Playing            53981 non-null  object 
 12  Backlogs           53981 non-null  object 
 13  Wishlist           53981 non-null  object 
dtypes: float64(1), object(13)
memory usage: 5.8+ MB


In [164]:
existing_data.head(150)

,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Platforms,Plays,Playing,Backlogs,Wishlist
0,Elden Ring,"Feb 25, 2022","['FromSoftware', 'Bandai Namco Entertainment']",4.5,6.2K,6.2K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","['👍', 'A good effort from konami, but at the e...","['Windows PC, PlayStation 4, Xbox One, PlaySta...",33K,5.2K,8.7K,7.3K
1,The Legend of Zelda: Tears of the Kingdom,"May 12, 2023","['Nintendo', 'Nintendo EPD Production Group No...",4.5,4.2K,4.2K,"['Adventure', 'RPG']",The Legend of Zelda: Tears of the Kingdom is t...,['Legend of Zelda: Tears of the Kingdom is a v...,['Nintendo Switch'],16K,6.5K,6.3K,6.9K
2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo EPD Production Group No. 3', 'Ninte...",4.4,6.6K,6.6K,"['Adventure', 'Puzzle', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,"['👍', 'very nice game, my first Legend of Zeld...","['Wii U, Nintendo Switch']",48K,3.7K,7.6K,3.9K
3,Hades,"Dec 07, 2018",['Supergiant Games'],4.3,4.2K,4.2K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,"[""I enjoyed it quite a bit, but with games lik...","['Windows PC, Mac, PlayStation 4, Xbox One, Pl...",35K,4.4K,10K,5.3K
4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,4.6K,4.6K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"['This game is perfect, I immediately fell in ...","['Windows PC, Mac, Wii U, Linux, Nintendo Swit...",35K,3.6K,13K,3.6K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Shadow of the Colossus,"Oct 18, 2005","['Sony Computer Entertainment', 'Team Ico']",4.4,2.2K,2.2K,"['Adventure', 'Platform', 'Puzzle']",An open-world action/adventure game in which a...,"['Lonely ass game', 'a humanidade não merece e...",['PlayStation 2'],15K,224,4.7K,2.6K
146,Disco Elysium,"Oct 15, 2019",['ZA/UM'],4.6,1.8K,1.8K,"['Adventure', 'RPG']","A CRPG in which, waking up in a hotel room a t...",['O meu mais novo jogo favorito! Disco Elysium...,"['Windows PC, Mac']",6.9K,838,5K,2.5K
147,Cult of the Lamb,"Aug 11, 2022","['Devolver Digital', 'Massive Monster']",3.6,981,981,"['Adventure', 'Brawler', 'Indie', 'RPG', 'Simu...",Cult of the Lamb casts players in the role of ...,"[""The game's got great visuals and sound desig...","['Windows PC, Mac, PlayStation 4, Xbox One, Pl...",6.7K,944,3.6K,3.6K
148,Danganronpa: Trigger Happy Havoc,"Nov 25, 2010","['Spike ChunSoft', 'NIS America']",3.5,1.4K,1.4K,"['Adventure', 'Point-and-Click', 'Visual Novel']","In ""Danganronpa"" you'll dive into a series of ...",['For once I managed to go into a game complet...,"['Windows PC, Android, Mac, Linux, iOS, PlaySt...",17K,408,2.8K,1K
